<a href="https://colab.research.google.com/github/SnehaSaragadam/Python/blob/main/Poetry_Classification_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Necessary Libraries

In [3]:
import os
from transformers import GPT2Tokenizer, GPT2Model
import torch
import pandas as pd
import numpy as np

# Feature Extraction

Load Tokenizer and Model

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Initializing Variables

In [5]:
# Define the folder containing the poems
poems_folder = "/content/drive/MyDrive/Datasets/My Dataset"

# Define the size of each segment (adjust this based on the length of your poems)
segment_size = 50

# Initialize lists to store features and labels
all_features = []
all_labels = []

# Check maximum sequence length supported by the model
max_sequence_length = model.config.max_position_embeddings

Preprocessing Data & Embedding data

In [6]:
# Iterate through each class folder
for class_folder in os.listdir(poems_folder):
    class_path = os.path.join(poems_folder, class_folder)
    if os.path.isdir(class_path):
        # Iterate through each poem file in the class folder
        for poem_file in os.listdir(class_path):
            poem_path = os.path.join(class_path, poem_file)
            with open(poem_path, "r", encoding="utf-8") as f:
                poem_text = f.read()

            # Tokenize the poem text
            tokens = tokenizer.tokenize(poem_text)
            token_ids = tokenizer.convert_tokens_to_ids(tokens)

            # Truncate sequences longer than max_sequence_length
            if len(token_ids) > max_sequence_length:
                token_ids = token_ids[:max_sequence_length]

            # Generate representations for each segment
            segment_embeddings = []
            for i in range(0, len(token_ids), segment_size):
                segment = token_ids[i:i+segment_size]
                segment_input = torch.tensor([segment])
                with torch.no_grad():
                    outputs = model(segment_input)
                    segment_embedding = torch.mean(outputs[0], dim=1).squeeze()
                    segment_embeddings.append(segment_embedding)

            # Concatenate segment embeddings into a single feature vector
            poem_features = torch.cat(segment_embeddings, dim=0).numpy()
            all_features.append(poem_features)
            all_labels.append(class_folder)  # Assuming folder name represents class label

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Datasets/My Dataset'

Extracted Features as .csv

In [ ]:
# Convert features and labels to DataFrames
features_df = pd.DataFrame(all_features)
labels_df = pd.DataFrame({"label": all_labels})

# Concatenate features and labels
poems_data = pd.concat([features_df, labels_df], axis=1)

# Save poem features to CSV file
poems_data.to_csv('poems_data.csv', index=False)

# KNN Classification

Mean of Each Class

In [ ]:
# Read the CSV file into a DataFrame
poems_data = pd.read_csv('poems_data.csv')

# Filter the DataFrame to include only the "Haiku" and "Sonnet" classes
haiku_data = poems_data[poems_data['label'] == 'Haiku']
sonnet_data = poems_data[poems_data['label'] == 'Sonnet']

# Calculate the mean of all features class-wise for "Haiku"
haiku_mean = np.mean(haiku_data.drop(columns=['label']), axis=0)

# Calculate the mean of all features class-wise for "Sonnet"
sonnet_mean = np.mean(sonnet_data.drop(columns=['label']), axis=0)

print("Mean features for Haiku class:")
print(haiku_mean)
print("\nMean features for Sonnet class:")
print(sonnet_mean)

Mean features for Haiku class:
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
16123   NaN
16124   NaN
16125   NaN
16126   NaN
16127   NaN
Length: 16128, dtype: float64

Mean features for Sonnet class:
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
16123   NaN
16124   NaN
16125   NaN
16126   NaN
16127   NaN
Length: 16128, dtype: float64
